In [1]:
%pip install igraph niquests pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import igraph as ig
import time
import itertools
import requests
import random
import asyncio
from niquests import AsyncSession, Response

pd.set_option("mode.copy_on_write", True)

In [3]:
import IPython
IPython.version_info

(7, 27, 0, '')

In [111]:
USE_PANDAS_PERF=False # if multiple CPU cores are available

In [11]:
lt_df = pd.read_pickle("data/fc_v3engagement_fid_2024-06-18_filtered_df.pkl")

In [12]:
lt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716438 entries, 7 to 3772376
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   i       716438 non-null  int64
 1   j       716438 non-null  int64
 2   v       716438 non-null  int64
dtypes: int64(3)
memory usage: 21.9 MB


In [13]:
lt_df.head()

,i,j,v
7,410568,525376,114
21,218142,516028,197
24,451888,349675,28
25,440212,3887,8
26,2282,2433,17


In [14]:
%%time
g = ig.Graph.Read_Pickle('data/fc_v3engagement_fid_2024-06-18_filtered_ig.pkl')

CPU times: user 506 ms, sys: 98.5 ms, total: 604 ms
Wall time: 1.05 s


In [15]:
g.summary()

'IGRAPH DN-- 17969 716438 -- \n+ attr: name (v), v (e)'

In [16]:
rainbow_fids = pd.read_csv('/tmp/rainbow_fids_18sep2024.csv')

In [17]:
rainbow_fids.head()

,fid
0,623997
1,20353
2,337715
3,327341
4,344026


In [18]:
rainbow_fids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22869 entries, 0 to 22868
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   fid     22869 non-null  int64
dtypes: int64(1)
memory usage: 178.8 KB


In [5]:
async def _get_direct_edges_df(
  fids,
  df,
  max_neighbors,
):
    # WARNING we are operating on a shared dataframe...
    # ...inplace=False by default, explicitly setting here for emphasis
    out_df = df[df['i'].isin(fids)].sort_values(by=['v'], ascending=False, inplace=False)[:max_neighbors]
    return out_df

In [174]:
async def find_vertex_idx(ig, fid):
    try:
        return ig.vs.find(name=fid).index
    except:
        return None

async def _fetch_korder_neighbors(
  fids,graph,max_degree,max_neighbors,min_degree = 1
):

    # vids = [find_vertex_idx(graph.graph, fid) for fid in fids]
    # vids = list(filter(None, vids)) # WARNING - this filters vertex id 0 also
    vids = [vid for fid in fids for vid in [await find_vertex_idx(graph, fid)] if vid is not None ]
    if len(vids) <= 0:
        raise Exception(f"{fids}:Invalid fids")
    try:
        klists = []
        mindist_and_order = min_degree
        limit = max_neighbors
        while mindist_and_order <= max_degree:
            neighbors = graph.neighborhood(
                vids, order=mindist_and_order, mode="out", mindist=mindist_and_order
            )
            # TODO prune the graph after sorting by edge weight
            klists.append(graph.vs[neighbors[0][:limit]]["name"])
            limit = limit - len(neighbors[0])
            if limit <= 0:
                break # we have reached limit of neighbors
            mindist_and_order += 1
        # end of while
        return set(itertools.chain(*klists))
    except ValueError:
        raise Exception(f"{fids}:Neighbors not found")

In [195]:
async def _get_neighbors_edges(
  fids, df, graph, max_degree, max_neighbors,
):

    start_time = time.perf_counter()
    neighbors_df = await _get_direct_edges_df(fids, df, max_neighbors)
    print(f"{fids}:direct_edges_df took {time.perf_counter() - start_time} secs"
          f" for {len(neighbors_df)} first degree edges")
    k_neighbors_list= neighbors_df['j'].unique().tolist()

    max_neighbors = max_neighbors - len(k_neighbors_list)
    if max_neighbors > 0 and max_degree > 1:

        start_time = time.perf_counter()
        k_list_next = await _fetch_korder_neighbors(fids, graph, max_degree, max_neighbors, min_degree=2)
        print(f"{fids}:{time.perf_counter() - start_time} secs for {len(k_list_next)} neighbors")

        start_time  = time.perf_counter()
        k_neighbors_list.extend(k_list_next)
        if USE_PANDAS_PERF:
            # if multiple CPU cores are available
            k_df = df.query('i in @k_neighbors_list').query('j in @k_neighbors_list')
        else:
            # filter with an '&' is slower because of the size of the dataframe
            # split the filtering so that indexes can be used if present
            # k_df = graph.df[graph.df['i'].isin(k_neighbors_list) & graph.df['j'].isin(k_neighbors_list)]
            k_df = df[df['i'].isin(k_neighbors_list)]
            k_df = k_df[k_df['j'].isin(k_neighbors_list)]
        # .loc will throw KeyError when fids have no outgoing actions
        ### in other words, some neighbor fids may not be present in 'i'
        # k_df = graph.df.loc[(k_neighbors_list, k_neighbors_list)]
        print(f"{fids}:k_df took {time.perf_counter() - start_time} secs for {len(k_df)} edges")

        start_time  = time.perf_counter()
        neighbors_df = pd.concat([neighbors_df, k_df])
        print(f"{fids}:neighbors_df concat took {time.perf_counter() - start_time} secs"
              f" for {len(neighbors_df)} edges")

    return neighbors_df

In [135]:
async def get_neighbors_list(
  fids, df, graph, max_degree = 2, max_neighbors = 100,
):
    df = await _get_neighbors_edges(fids, df, graph, max_degree, max_neighbors)
    # WARNING we are operating on a shared dataframe...
    # ...inplace=False by default, explicitly setting here for emphasis
    out_df = df.groupby(by='j')[['v']].sum().sort_values(by=['v'], ascending=False, inplace=False)
    return out_df.index.to_list()

In [136]:
# input_fid=189237
input_fid=381376
max_degree=5
max_neighbors=1000

In [176]:
await _get_direct_edges_df(df=lt_df, fids=[input_fid], max_neighbors=max_neighbors)

,i,j,v
2024880,381376,602,12
2024881,381376,20909,12
2427621,381376,205077,1


In [196]:
n_df = await _get_neighbors_edges(df=lt_df, graph=g, 
                                  fids=[input_fid], max_degree=max_degree, max_neighbors=max_neighbors)

[381376]:direct_edges_df took 0.033938927999770385 secs for 3 first degree edges
[381376]:0.030465339999864227 secs for 997 neighbors
[381376]:k_df took 0.08426359099939873 secs for 173277 edges
[381376]:neighbors_df concat took 0.011236893000386772 secs for 173280 edges


In [197]:
n_df.head(5)

,i,j,v
2024880,381376,602,12
2024881,381376,20909,12
2427621,381376,205077,1
21,218142,516028,197
25,440212,3887,8


In [198]:
n_df[n_df['i']== 381376]

,i,j,v
2024880,381376,602,12
2024881,381376,20909,12
2427621,381376,205077,1


In [199]:
lt_df[lt_df['i']== 602]

,i,j,v
8939,602,2602,1331
16287,602,3887,37
17968,602,207,1
18903,602,17507,2
20322,602,366,1
...,...,...,...
2065384,602,24235,1
2065385,602,194372,1
2065390,602,238132,1
2065404,602,658756,1


In [200]:
2602 in n_list and 602 in n_list

True

In [201]:
n_df[(n_df['i']== 602) & (n_df['j']== 2602)]

,i,j,v
8939,602,2602,1331


In [202]:
lt_df[(lt_df['i']== 602) & (lt_df['j']== 2602)]

,i,j,v
8939,602,2602,1331


In [203]:
n_list = await get_neighbors_list(df=lt_df, graph=g, 
                                 fids=[input_fid], max_degree=max_degree, max_neighbors=max_neighbors)

[381376]:direct_edges_df took 0.04193130800013023 secs for 3 first degree edges
[381376]:0.007435030998749426 secs for 997 neighbors
[381376]:k_df took 0.031804426998860436 secs for 173277 edges
[381376]:neighbors_df concat took 0.0015092829999048263 secs for 173280 edges


In [204]:
len(n_list)

1000

In [205]:
n_list[:10]

[602, 403619, 234616, 3, 20909, 2904, 263574, 408746, 436577, 539]

In [207]:
async def go_eigentrust(
    pretrust, max_pt_id, localtrust, max_lt_id,
):
    start_time = time.perf_counter()

    lt_len_before = len(localtrust)
    localtrust[:] = (x for x in localtrust if x["i"] != x["j"])
    lt_len_after = len(localtrust)
    if lt_len_before != lt_len_after:
        print(f"dropped {lt_len_before-lt_len_after} records with i == j")

    req = {
        "pretrust": {
            "scheme": "inline",
            # "size": int(max_pt_id)+1, #np.int64 doesn't serialize; cast to int
            "size": max_pt_id,
            "entries": pretrust,
        },
        "localTrust": {
            "scheme": "inline",
            # "size": int(max_lt_id)+1, #np.int64 doesn't serialize; cast to int
            "size": max_lt_id,
            "entries": localtrust,
        },
        "alpha": 0.5,
    }

    async with AsyncSession() as s:
        response = await s.post(
            "http://localhost:8080/basic/v1/compute",
            json=req,
            headers={"Accept": "application/json", "Content-Type": "application/json"},
            timeout=3000,
        )

        if response.status_code != 200:
            print(f"Server error: {response.status_code}:{response.reason}")
            raise Exception("Unknown error")
        trustscores = response.json()["entries"]
        print(
            f"{pretrust}:eigentrust took {time.perf_counter() - start_time} secs for {len(trustscores)} scores"
        )
        return trustscores

In [210]:
async def get_neighbors_scores(
    fids, df, graph, max_degree, max_neighbors
):
    start_time = time.perf_counter()
    try:
        df = await _get_neighbors_edges(fids, df, graph, max_degree, max_neighbors)
        print(
            f"{fids}:dataframe took {time.perf_counter() - start_time} secs for {len(df)} edges"
        )
    except Exception as e:
        print(e)
        return []

    if df.shape[0] < 1:
        return []
    
    stacked = df.loc[:, ("i", "j")].stack()
    pseudo_id, orig_id = stacked.factorize()
    
    # pseudo_df is a new dataframe to avoid modifying existing shared global df
    pseudo_df = pd.Series(pseudo_id, index=stacked.index).unstack()
    pseudo_df.loc[:, ("v")] = df.loc[:, ("v")]

    if len(fids) > 1:
        # when more than 1 fid in input list, the neighbor edges may not have some input fids.
        pt_fids = orig_id.where(orig_id.isin(fids))
    else:
        pt_fids = fids
    pt_len = len(fids)
    # pretrust = [{'i': fid, 'v': 1/pt_len} for fid in pt_fids]
    pretrust = [
        {"i": orig_id.get_loc(fid), "v": 1 / pt_len}
        for fid in pt_fids
        if not np.isnan(fid)
    ]
    # max_pt_id = max(pt_fids)
    max_pt_id = len(orig_id)

    localtrust = pseudo_df.to_dict(orient="records")
    # max_lt_id = max(df['i'].max(), df['j'].max())
    max_lt_id = len(orig_id)

    print(
        f"{fids}:max_lt_id:{max_lt_id}, localtrust size:{len(localtrust)},"
        f" max_pt_id:{max_pt_id}, pretrust size:{len(pretrust)}"
    )

    i_scores = await go_eigentrust(
        pretrust=pretrust,
        max_pt_id=max_pt_id,
        localtrust=localtrust,
        max_lt_id=max_lt_id,
    )

    # rename i and v to fid and score respectively
    # also, filter out input fids
    fid_scores = [
        {"fid": int(orig_id[score["i"]]), "score": score["v"]}
        for score in i_scores
        if score["i"] not in fids
    ]
    
    fid_scores = sorted(fid_scores, key=lambda d: d['score'], reverse=True)
    print(
        f"{fids}:sample fid_scores:{random.sample(fid_scores, min(10, len(fid_scores)))}"
    )
    return fid_scores


In [211]:
n_scores = await get_neighbors_scores(df=lt_df, graph=g, 
                                     fids=[input_fid], max_degree=max_degree, max_neighbors=max_neighbors)

[381376]:direct_edges_df took 0.020950235000782413 secs for 3 first degree edges
[381376]:0.005826417000207584 secs for 997 neighbors
[381376]:k_df took 0.03734304900171992 secs for 173277 edges
[381376]:neighbors_df concat took 0.0022968869998294394 secs for 173280 edges
[381376]:dataframe took 0.07672896799886075 secs for 173280 edges
[381376]:max_lt_id:1001, localtrust size:173280, max_pt_id:1001, pretrust size:1
[{'i': 0, 'v': 1.0}]:eigentrust took 1.2604352609996567 secs for 1001 scores
[381376]:sample fid_scores:[{'fid': 230238, 'score': 0.0012582710890309294}, {'fid': 404134, 'score': 1.92863005589967e-05}, {'fid': 18489, 'score': 3.816718517881292e-05}, {'fid': 13267, 'score': 6.732914020169019e-05}, {'fid': 1350, 'score': 7.782973942198193e-05}, {'fid': 268290, 'score': 3.0359372222601847e-05}, {'fid': 418296, 'score': 1.939808943267957e-05}, {'fid': 406493, 'score': 5.2037046154882755e-05}, {'fid': 698923, 'score': 0.00010083878546331906}, {'fid': 1841, 'score': 0.00011478401

In [212]:
n_scores

[{'fid': 381376, 'score': 0.500000001023698},
 {'fid': 602, 'score': 0.17559039408033603},
 {'fid': 20909, 'score': 0.1289815801706022},
 {'fid': 205077, 'score': 0.010115066267820935},
 {'fid': 3, 'score': 0.008118921870795479},
 {'fid': 446697, 'score': 0.005740434554752834},
 {'fid': 211635, 'score': 0.00518981646206328},
 {'fid': 2602, 'score': 0.00512280428421076},
 {'fid': 234616, 'score': 0.004306701352898443},
 {'fid': 403619, 'score': 0.003917909065123692},
 {'fid': 2904, 'score': 0.0028655952321926122},
 {'fid': 253127, 'score': 0.00242687954418881},
 {'fid': 8446, 'score': 0.00223879759375511},
 {'fid': 680, 'score': 0.0021495060299025507},
 {'fid': 408746, 'score': 0.0019149395049368762},
 {'fid': 534, 'score': 0.0018790611001031929},
 {'fid': 20919, 'score': 0.0017269885055593},
 {'fid': 3642, 'score': 0.0017130935055088496},
 {'fid': 354795, 'score': 0.001708552636641321},
 {'fid': 426045, 'score': 0.0016880115058467004},
 {'fid': 366713, 'score': 0.001581156908800773},
 

In [74]:
async def main() -> None:
    tasks = []
    for fid in rainbow_fids['fid'].sample(5):
        print(fid)
        tasks.append(
            asyncio.create_task(
                get_neighbors_scores(
                    fids=[fid], df=lt_df, graph=g, max_degree=5, max_neighbors=1000)))
    responses = await asyncio.gather(*tasks)

In [75]:
import timeit
start_time = timeit.default_timer()
await main()
elapsed = timeit.default_timer() - start_time
print(f"Overall time taken: {elapsed} secs")

381376
401911
378112
249346
447671
[381376]:direct_edges_df took 0.02269602499973189 secs for 3 first degree edges
[381376]:0.021266510999794264 secs for 997 neighbors
[381376]:k_df took 0.08118097099986699 secs for 170780 edges
[381376]:neighbors_df concat took 0.01537540200024523 secs for 170783 edges
[381376]:dataframe took 0.1554080440000689 secs for 170783 edges
[381376]:max_lt_id:1001, localtrust size:170783, max_pt_id:1001, pretrust size:1
[401911]:direct_edges_df took 0.009839229999670351 secs for 0 first degree edges
[401911]:0.0016770669999459642 secs for 0 neighbors
[401911]:k_df took 0.035437240000192105 secs for 0 edges
[401911]:neighbors_df concat took 0.0011912800000573043 secs for 0 edges
[401911]:dataframe took 0.04958241900021676 secs for 0 edges
[378112]:direct_edges_df took 0.010593187999802467 secs for 0 first degree edges
[378112]:Invalid fids
[249346]:direct_edges_df took 0.006716549999964627 secs for 3 first degree edges
[249346]:0.007718137999745522 secs for 99